# CMIP5 build intake catalog

Build intake catalog for CMIP5 from manifest files.

In [ ]:
import pandas as pd

## Load manifest

In [ ]:
df = pd.read_csv("../cmip5/manifest_C3S-34a-Lot1_cmip5-monthly-single-level_http_latest.txt.gz", names=["url"])
df

In [ ]:
df.loc[0]["url"]

## Build new dataframe

In [ ]:
df_new = pd.DataFrame(columns = ['drs_id', 'path'])
df_new

In [ ]:
df_new["path"] = df["url"].str.replace("http://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cmip5/", "", regex=False)
df_new

In [ ]:
# c3s-cmip5.output1.MPI-M.MPI-ESM-LR.historical.mon.atmos.Amon.r1i1p1.tas.v20120315

In [ ]:
df_new["drs_id"] = df_new["path"].apply(lambda x: 'c3s-cmip5.' + '.'.join(x.split('/')[0:-1]))
df_new

In [ ]:
cat_name = f"c3s-cmip5_v20220421.csv.gz"
cat_path = cat_name

df_new.to_csv(cat_path, index=False, compression="gzip")

 ## Load catalog

In [ ]:
df = pd.read_csv(cat_path)
df